# Python Client Use

This notebook demonstrates simple use of this plugin via the Steamship client.

In [5]:
# NOTE!
# You will likely want to set your own STEAMSHIP_PROFILE here, or change it to None!
STEAMSHIP_PROFILE="staging"

# This should match the value found in the `handle` field of `steamship.json`
PLUGIN_HANDLE="audio-to-text-blockifier-default"

In [10]:
# Create our client.
from steamship import Steamship, PluginInstance, File, Block, MimeTypes
ship = Steamship(profile=STEAMSHIP_PROFILE)

print(f"You are operating within workspace: {ship.config.space_handle or 'default'}")

You are operating within workspace: default


In [34]:
from os import path

# Upload a file
file_path = path.join('..', 'test', 'data', 'small_utterances.m4a')
file = File.create(ship, filename=file_path, mime_type=MimeTypes.MP3).data

In [35]:
# Create a blockifier to parse the CSV file
blockifier = PluginInstance.create(ship, plugin_handle=PLUGIN_HANDLE).data

print(f"Your plugin handle is: {blockifier.handle}")

Your plugin handle is: sassy-meadow-9kfbu


In [36]:
blockify_task = file.blockify(blockifier.handle)
print(f"Your blockify task ID is {blockify_task.task.task_id}")

Your blockify task ID is 6F74C472-C454-409A-8201-A7024F35031D


In [37]:
# All plugin operations are asynchronous.
# Calling .wait() will wait until a timeout is reached or until we have a result
blockify_task.wait()

In [38]:
blockify_task.task

Task(client=Steamship(config=Configuration(api_key='05734E32-F33B-49E0-BD78-FB75F5F36B24', api_base=HttpUrl('https://api.staging.steamship.com/api/v1/', scheme='https', host='api.staging.steamship.com', tld='com', host_type='domain', port='443', path='/api/v1/'), app_base=HttpUrl('https://apps.staging.steamship.com/', scheme='https', host='apps.staging.steamship.com', tld='com', host_type='domain', port='443', path='/'), web_base=HttpUrl('https://app.staging.steamship.com/', scheme='https', host='app.staging.steamship.com', tld='com', host_type='domain', port='443', path='/'), space_id='CB587FFD-62AD-480D-96C6-5FE89AC9C7EA', space_handle='default', profile='staging')), task_id='6F74C472-C454-409A-8201-A7024F35031D', user_id='D12B7EDA-6631-43DE-A1A2-F814C3F3848B', space_id='CB587FFD-62AD-480D-96C6-5FE89AC9C7EA', input='{"id":"218897CE-059D-4691-9EC1-71066762F9DD","type":"file","pluginInstance":"sassy-meadow-9kfbu"}', output='{"file":{"blocks":[{"tags":[{"kind":"speaker","endIdx":70,"nam

In [40]:
for block in blockify_task.data.file.blocks:
    print(block.text)
    for tag in block.tags:
        print(f"- {tag.kind} / {tag.name}")

Hello. I love hot coffee. I hate cold coffee. I'd like to go to Paris.
- speaker / A
- timestamp / Hello.
- timestamp / I
- timestamp / love
- timestamp / hot
- timestamp / coffee.
- timestamp / I
- timestamp / hate
- timestamp / cold
- timestamp / coffee.
- timestamp / I'd
- timestamp / like
- timestamp / to
- timestamp / go
- timestamp / to
- timestamp / Paris.
- topic / Food&Drink>DiningOut
- topic / Food&Drink>WorldCuisines
- topic / BusinessAndFinance>Industries>HospitalityIndustry
- topic / Food&Drink
- topic / Travel>TravelLocations>SouthAmericaTravel
- topic / BusinessAndFinance>Economy>GasolinePrices
- topic / Automotive>AutoType>ConceptCars
- topic / BooksAndLiterature>TravelBooks
- topic / Food&Drink>Non-AlcoholicBeverages
- topic / EventsAndAttractions>Bars&Restaurants
- topic_summary / Food&Drink>DiningOut
- topic_summary / Food&Drink>WorldCuisines
- topic_summary / BusinessAndFinance>Industries>HospitalityIndustry
- topic_summary / Food&Drink
- topic_summary / Travel>Trav

In [ ]:
# Query - Get just the spam
spam_blocks = Block.query(client=ship, tag_filter_query='blocktag and name "spam"').data.blocks
print("SPAM\n====")
for block in spam_blocks:
    print(f"- {block.text}")